# 🍓 Task 3: Matching Strawberry to Peduncle

This notebook trains a **Siamese Network** to determine if a specific Strawberry belongs to a specific Peduncle (stem).
This is crucial for robotics to know which stem to cut for a target fruit.

**Method:**
1.  **Download Dataset** (Github Releases).
2.  **Generate Pairs**: Create positive pairs (True Parent) and negative pairs (Random Other Peduncle).
3.  **Siamese Model**: Shared Encoder + Contrastive/Binary Head.
4.  **Training**: Minimize Binary Cross Entropy.

## 1. Setup & Download

Standard setup block.

In [ ]:
import os
import sys
import json
import requests
import zipfile
import shutil
import glob
import inspect
import torch
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image
from tqdm.auto import tqdm
from pathlib import Path
import cv2
import random
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, models
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
    import timm


# --- Robust Dataset Configuration ---
VERSION_TAG = "Dataset"
BASE_URL = f"https://github.com/SergKurchev/strawberry_synthetic_dataset/releases/download/{VERSION_TAG}"
FILES_TO_DOWNLOAD = [
    "strawberry_dataset.zip.001",
    "strawberry_dataset.zip.002",
    "strawberry_dataset.zip.003"
]
OUTPUT_ZIP = "strawberry_dataset.zip"

def reconstruct_metadata(dataset_root):
    """Reconstructs depth_metadata.json from individual files in metadata_temp/"""
    print("⚠️ 'depth_metadata.json' not found. Attempting reconstruction from 'metadata_temp/'...")
    temp_dir = dataset_root / "metadata_temp"
    if not temp_dir.exists():
        print(f"❌ metadata_temp directory not found at {temp_dir}")
        return False

    combined_metadata = {}
    json_files = list(temp_dir.glob("*_meta.json"))
    print(f"  Found {len(json_files)} metadata chunks.")
    
    for json_file in tqdm(json_files, desc="Reconstructing Metadata"):
        try:
            # Filename format: 00001_meta.json -> corresponds to 00001.png
            # We assume the content of the json is the metadata dict for that image
            img_id = json_file.name.replace("_meta.json", "")
            img_name = f"{img_id}.png"
            
            with open(json_file, 'r') as f:
                data = json.load(f)
                combined_metadata[img_name] = data
        except Exception as e:
            print(f"  Warning: Failed to read {json_file}: {e}")

    if not combined_metadata:
        print("❌ Failed to reconstruct any metadata.")
        return False

    target_path = dataset_root / "depth_metadata.json"
    print(f"💾 Saving reconstructed metadata to {target_path}...")
    with open(target_path, 'w') as f:
        json.dump(combined_metadata, f, indent=2)
        
    return True

def setup_dataset():
    # 1. Search for existing dataset
    print("🔍 Searching for existing dataset...")
    
    # Helper to validate a root candidate
    def validate_root(p):
        if (p / "depth_metadata.json").exists():
            return True
        if (p / "metadata_temp").exists():
            # Try to fix it
            return reconstruct_metadata(p)
        return False

    # Recursive search in current dir
    for root, dirs, files in os.walk(".", topdown=True):
        p = Path(root)
        # Start optimization: don't go too deep or into hidden dirs
        if ".git" in p.parts or "temp_download" in p.parts:
            continue
            
        if "images" in dirs and ("depth_metadata.json" in files or "metadata_temp" in dirs):
            if validate_root(p):
                print(f"✅ Dataset found/Fixed at: {p}")
                return p

    # Check standard paths
    search_paths = [
        Path("strawberry_dataset"),
        Path("dataset/strawberry_dataset"),
        Path("/kaggle/input/last-straw-dataset/strawberry_dataset"),
        Path("/kaggle/input/strawberry_synthetic_dataset/strawberry_dataset")
    ]
    for p in search_paths:
        if p.exists():
            if validate_root(p):
                print(f"✅ Dataset found/Fixed at: {p}")
                return p

    print("⬇️ Dataset not found. Downloading from GitHub Releases...")
    
    # 2. Prepare Download Directory
    if os.path.exists("temp_download"):
        shutil.rmtree("temp_download")
    os.makedirs("temp_download", exist_ok=True)
    
    if os.path.exists(OUTPUT_ZIP):
        os.remove(OUTPUT_ZIP)

    # 3. Download and Combine
    with open(OUTPUT_ZIP, 'wb') as outfile:
        for filename in FILES_TO_DOWNLOAD:
            file_path = Path("temp_download") / filename
            url = f"{BASE_URL}/{filename}"
            
            print(f"  Downloading {filename} from {url}...")
            r = requests.get(url, stream=True)
            if r.status_code != 200:
                raise RuntimeError(f"Download failed for {filename}: HTTP {r.status_code}")
            
            with open(file_path, 'wb') as f:
                for chunk in r.iter_content(chunk_size=8192):
                    f.write(chunk)
            
            file_size_mb = file_path.stat().st_size / 1024 / 1024
            print(f"  Downloaded {filename} ({file_size_mb:.2f} MB). Appending to zip...")
            
            with open(file_path, 'rb') as infile:
                shutil.copyfileobj(infile, outfile)

    # 4. Extract
    total_size_mb = os.path.getsize(OUTPUT_ZIP)/1024/1024
    print(f"📂 Extracting {OUTPUT_ZIP} ({total_size_mb:.2f} MB)...")
    
    try:
        with zipfile.ZipFile(OUTPUT_ZIP, 'r') as zip_ref:
            zip_ref.extractall(".")
            print("  Extraction complete.")
    except zipfile.BadZipFile as e:
        print(f"❌ BadZipFile Error: {e}")
        raise e
    
    shutil.rmtree("temp_download", ignore_errors=True)
    if os.path.exists(OUTPUT_ZIP):
        os.remove(OUTPUT_ZIP)

    # --- FIX: Handle potential backslash filenames on Linux ---
    print("🧹 Checking for backslash issues in filenames...")
    count = 0
    # Iterate over files in current directory to check for backslashes in names
    for filename in os.listdir("."):
        if "\\" in filename:
            # It's a file with backslashes in name, implying flattened structure
            new_path = filename.replace("\\", "/") # standardize to forward slash
            
            # Create parent dirs
            parent = os.path.dirname(new_path)
            if parent:
                os.makedirs(parent, exist_ok=True)
            
            # Move file
            try:
                shutil.move(filename, new_path)
                count += 1
            except Exception as e:
                print(f"  Failed to move {filename} -> {new_path}: {e}")
            
    if count > 0:
        print(f"✅ Fixed {count} filenames with backslashes. Directory structure restored.")
        
    # 5. Locate and Fix
    print("🔎 Locating dataset root...")
    for root, dirs, files in os.walk(".", topdown=True):
        p = Path(root)
        if "images" in dirs and ("depth_metadata.json" in files or "metadata_temp" in dirs):
            if validate_root(p):
                 print(f"✅ Dataset extracted and verified at: {p}")
                 return p
            
    return None

DATASET_PATH = setup_dataset()
if not DATASET_PATH: raise RuntimeError("Dataset setup failed: Could not locate or reconstruct metadata")
DATASET_ROOT = DATASET_PATH


## 2. Generate Pairs

We need pairs of images (Strawberry, Peduncle) and a label (1=Match, 0=No Match).

In [ ]:
with open(DATASET_PATH / "annotations.json", 'r') as f: coco = json.load(f)

# Organize data
img_to_anns = defaultdict(list)
for ann in coco['annotations']:
    img_to_anns[ann['image_id']].append(ann)

pairs = []

CROPS_BASE = Path("matching_crops")
CROPS_BASE.mkdir(exist_ok=True)
(CROPS_BASE / "straw").mkdir(exist_ok=True)
(CROPS_BASE / "ped").mkdir(exist_ok=True)

print("🖇️ Generating Pairs...")

for img_info in tqdm(coco['images']):
    img_id = img_info['id']
    if img_id not in img_to_anns: continue

    img_path = DATASET_PATH / "images" / img_info['file_name']
    if not img_path.exists(): continue
    img_cv = cv2.imread(str(img_path))
    if img_cv is None: continue

    anns = img_to_anns[img_id]
    
    # Extract all objects first
    straws = []
    peds = [] # Dict: id -> path

    for ann in anns:
        x, y, w, h = [int(v) for v in ann['bbox']]
        # Pad
        pad=5
        x, y = max(0, x-pad), max(0, y-pad)
        w, h = min(img_cv.shape[1]-x, w+2*pad), min(img_cv.shape[0]-y, h+2*pad)
        crop = img_cv[y:y+h, x:x+w]
        if crop.size==0: continue
        
        save_name = f"{img_id}_{ann['id']}.jpg"
        
        if ann['category_id'] in [0, 1, 2]: # Strawberry
            p = CROPS_BASE / "straw" / save_name
            cv2.imwrite(str(p), crop)
            straws.append({"ann": ann, "path": p})
        elif ann['category_id'] == 3: # Peduncle
            p = CROPS_BASE / "ped" / save_name
            cv2.imwrite(str(p), crop)
            peds.append({"ann": ann, "path": p})

    # Make positive pairs
    ped_map = {p['ann']['instance_id']: p for p in peds}
    
    for s in straws:
        parent_id = s['ann'].get('parent_id', 0)
        if parent_id in ped_map:
            # Positive
            pairs.append((str(s['path']), str(ped_map[parent_id]['path']), 1.0))
            
            # Negative (Hard mining: same image, different peduncle)
            other_peds = [pid for pid in ped_map.keys() if pid != parent_id]
            if other_peds:
                neg_pid = random.choice(other_peds)
                pairs.append((str(s['path']), str(ped_map[neg_pid]['path']), 0.0))

print(f"✅ Generated {len(pairs)} pairs.")

## 3. Siamese Network Training

Dual-input network.

In [ ]:
# Dataset
class SiameseDataset(Dataset):
    def __init__(self, pairs, transform=None):
        self.pairs = pairs
        self.transform = transform
        
    def __len__(self): return len(self.pairs)
    
    def __getitem__(self, idx):
        p1, p2, label = self.pairs[idx]
        img1 = Image.open(p1).convert("RGB")
        img2 = Image.open(p2).convert("RGB")
        if self.transform:
            img1 = self.transform(img1)
            img2 = self.transform(img2)
        return img1, img2, torch.tensor(label, dtype=torch.float32)

# Model
class SiameseNet(nn.Module):
    def __init__(self):
        super().__init__()
        # Shared Encoder
        self.encoder = timm.create_model('resnet18', pretrained=True, num_classes=0)
        
        # Classification Head
        self.head = nn.Sequential(
            nn.Linear(512 * 2, 256),
            nn.ReLU(),
            nn.Dropout(0.2),
            nn.Linear(256, 1),
            nn.Sigmoid()
        )
        
    def forward(self, x1, x2):
        f1 = self.encoder(x1)
        f2 = self.encoder(x2)
        combined = torch.cat([f1, f2], dim=1)
        return self.head(combined).squeeze()

# Setup
train_pairs, val_pairs = train_test_split(pairs, test_size=0.2, random_state=42)

tf = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

train_loader = DataLoader(SiameseDataset(train_pairs, tf), batch_size=32, shuffle=True)
val_loader = DataLoader(SiameseDataset(val_pairs, tf), batch_size=32, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SiameseNet().to(device)
optimizer = optim.Adam(model.parameters(), lr=1e-4)
criterion = nn.BCELoss()

print("🚀 Training Siamese Network...")
for epoch in range(5):
    model.train()
    loss_acc = 0
    for x1, x2, y in tqdm(train_loader, desc=f"Epoch {epoch+1}"):
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        optimizer.zero_grad()
        out = model(x1, x2)
        loss = criterion(out, y)
        loss.backward()
        optimizer.step()
        loss_acc += loss.item()
    print(f"  Loss: {loss_acc/len(train_loader):.4f}")

print("✅ Training Complete")

## 4. Evaluate

Check accuracy.

In [ ]:
model.eval()
correct = 0
total = 0
with torch.no_grad():
    for x1, x2, y in val_loader:
        x1, x2, y = x1.to(device), x2.to(device), y.to(device)
        out = model(x1, x2)
        preds = (out > 0.5).float()
        correct += (preds == y).sum().item()
        total += y.size(0)

print(f"🎯 Validation Accuracy: {correct/total*100:.2f}%")